## AIHub Json Preprocessing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [1]:
import re
import os
import kss
import ray
import json
import time
from time import sleep
from tqdm import tqdm
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [2]:
pwd

'D:\\AIHUB'

### AIHUB 행정 문서 대상 기계독해 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=569)

#### Convert JSON File to TXT File

In [ ]:
from data_preprocessing import make_train_valid_json_txt_file_path_list
from data_preprocessing import divide_source_file_list
from extract_source_text import make_sources

In [13]:
json_path_list = ['AIHUB_행정 문서 대상 기계독해 데이터/Training/'+ '/**/*.json', 
                  'AIHUB_행정 문서 대상 기계독해 데이터/Validation/'+ '/**/*.json']
txt_path_list = ["exploration/machine_reading_data_for_administrative_documents_pro/AIHUB_machine_reading_data_for_administrative_documents_train_", 
                 "exploration/machine_reading_data_for_administrative_documents_pro/AIHUB_machine_reading_data_for_administrative_documents_valid_"]

In [14]:
train_json_file_list, valid_json_file_list, train_txt_file_path_list, valid_txt_file_path_list = \
    make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list)

The number of file: 12


In [15]:
source_file_index_df = pd.DataFrame(train_json_file_list, columns=['source_file_name'])
source_file_index_df.to_excel("source_file_index/machine_reading_data_for_administrative_documents_source_train_file_index.xlsx", index=False)

source_file_index_df = pd.DataFrame(valid_json_file_list, columns=['source_file_name'])
source_file_index_df.to_excel("source_file_index/machine_reading_data_for_administrative_documents_source_valid_file_index.xlsx", index=False)

In [19]:
def count_number_of_txt_file_with_batch_list(source_file_list, batch_size):
    
    source_file_by_batch_df = pd.DataFrame({'File':[0], 'Length of Source List':[0],
                                            'The Number of TXT File':[0], 
                                            'Description':[0]})
                                            
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    
    for i in range(len(source_file_list)):    
        
        source_file = source_file_list[i]        

        with open(source_file, 'r', encoding='utf-8') as one_json_file:
            one_json_sample = json.load(one_json_file) 

        source_list = make_sources(one_json_sample)
        
        the_number_of_txt_file = ((len(source_list) // batch_size) + 1) 

        if len(source_list) >= batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file, ""]
            the_number_of_txt_file_list.append(the_number_of_txt_file)
            the_number_of_total_txt_file  += the_number_of_txt_file

        elif len(source_list) < batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file,
                                              "not subject of batch. small source list."]
            the_number_of_txt_file_list.append(1)
            the_number_of_total_txt_file  += 1

    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    if 'rain' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/machine_reading_data_for_administrative_documents_train.xlsx", index=False)
    elif 'alid' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/machine_reading_data_for_administrative_documents_valid.xlsx", index=False)
    else:
         source_file_by_batch_df.to_excel("source_file_by_batch/machine_reading_data_for_administrative_documents_text.xlsx", index=False)
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [20]:
def write_jsontext_to_txt_file_with_batch_list(source_file_list,
                                    text_file_path_list,
                                    batch_size, the_number_of_txt_file_list):

  progress_length = sum(the_number_of_txt_file_list)
  print("[Size]")
  print("The number of preprocessing corpus: " + str(progress_length))
  print("\n[Order]")
  pbar = tqdm(range(progress_length))
  num = 0
  for i in range(len(source_file_list)):

    source_file = source_file_list[i]
    
    with open(source_file, 'r', encoding='utf-8') as one_json_file:
      one_json_sample = json.load(one_json_file)
      
    source_list = make_sources(one_json_sample)
    
    n = batch_size
    source_batch_list = list(divide_source_file_list(source_list, n))
    
    for source_list in source_batch_list:
        
        if 'tableqa' in source_file:

            with open(os.path.join('AIHUB_corpus/' + text_file_path_list[i][:-4] + "_" + str(num) + "_tableqa.txt"), "a", encoding='utf-8') as fp:        
                fp.write("\n".join(source_list))   
            num += 1  
            pbar.n += 1
            pbar.refresh()
            time.sleep(0.01)
  
        else:

            with open(os.path.join('AIHUB_corpus/' + text_file_path_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
                fp.write("\n".join(source_list))   

            num += 1  
            pbar.n += 1
            pbar.refresh()
            time.sleep(0.01)
  pbar.close()  

In [21]:
batch_size = 10000
the_number_of_train_txt_file, the_number_of_train_txt_file_list = count_number_of_txt_file_with_batch_list(train_json_file_list, batch_size)

Batch Size: 10000
The number of txt file: 42


In [22]:
source_file_by_batch_train_df = pd.read_excel('source_file_by_batch/machine_reading_data_for_administrative_documents_train.xlsx', engine='openpyxl')  
source_file_by_batch_train_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_tabl...,125814,13,NaN
1,1,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_unan...,19656,2,NaN
2,2,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_mult...,30170,4,NaN
3,3,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_span...,127864,13,NaN
4,4,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_text...,33744,4,NaN
5,5,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_span...,58148,6,NaN


In [23]:
batch_size = 10000
write_jsontext_to_txt_file_with_batch_list(train_json_file_list, train_txt_file_path_list,
                batch_size, the_number_of_train_txt_file_list)

[Size]
The number of preprocessing corpus: 42

[Order]


100%|██████████| 42/42 [00:26<00:00,  1.61it/s]


In [24]:
batch_size = 10000
the_number_of_valid_txt_file, the_number_of_valid_txt_file_list = count_number_of_txt_file_with_batch_list(valid_json_file_list, batch_size)

Batch Size: 10000
The number of txt file: 11


In [25]:
source_file_by_batch_valid_df = pd.read_excel('source_file_by_batch/machine_reading_data_for_administrative_documents_valid.xlsx', engine='openpyxl')  
source_file_by_batch_valid_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_ta...,23578,3,NaN
1,1,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_un...,3686,1,not subject of batch. small source list.
2,2,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_mu...,5626,1,not subject of batch. small source list.
3,3,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_sp...,23970,3,NaN
4,4,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_te...,6308,1,not subject of batch. small source list.
5,5,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_sp...,10916,2,NaN


In [26]:
batch_size = 10000
write_jsontext_to_txt_file_with_batch_list(valid_json_file_list, valid_txt_file_path_list,
                batch_size, the_number_of_valid_txt_file_list)

[Size]
The number of preprocessing corpus: 11

[Order]


100%|██████████| 11/11 [00:06<00:00,  1.69it/s]


#### Preprocess TXT File

In [ ]:
from sentence_segmentation import preprocessing_text
from data_preprocessing import make_pro_post_txt_file_path_list
from data_preprocessing import merge_and_deduplicate_corpus_txt
from reading_data import reading_txt

In [ ]:
pro_corpus_path = "AIHUB_corpus/exploration/machine_reading_data_for_administrative_documents_pro/AIHUB_machine_reading_data_for_administrative_documents_train_" + "*.txt"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_path_list(pro_corpus_path)

In [ ]:
len(pro_total_corpus_path_list)

In [ ]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "source"

reading_txt(pro_coprus_file, line_length, data_type)

In [ ]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "preprocessing"

reading_txt(pro_coprus_file, line_length, data_type)

In [ ]:
ray.init(num_cpus = 4)

@ray.remote
def ray_preprocessing_text(source, corpus_path):

    preprocessing_sentence_list = preprocessing_text(source, corpus_path)

    return preprocessing_sentence_list

In [ ]:
def preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list):

    progress_length = len(pro_total_corpus_path_list)   
    print("[Size]")
    print("The number of preprocessing corpus: " + str(len(pro_total_corpus_path_list)))
    print("\n[Order]")
    pbar = tqdm(range(progress_length))
    num = 0
    process_num = 10    

    for pro, post in zip(pro_total_corpus_path_list, post_total_corpus_path_list):
        
        sentence_list = []
        
        with open(pro, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines() 
            nested_lines_num = len(lines) // process_num
            for i in range(nested_lines_num - 1):
                start_line = process_num * i
                end_line = process_num * (i+1)
                
                if 'tableqa' in pro:
                    futures = [ray_preprocessing_text.remote(lines[start_line:end_line][j], pro) for j in range(process_num)]
                    results = ray.get(futures)

                    if i == nested_lines_num - 2:
                        futures = [ray_preprocessing_text.remote(lines[end_line:][j], pro) for j in range(len(lines) - end_line)]
                        results = ray.get(futures)
                        
                else:
                    futures = [ray_preprocessing_text.remote(lines[start_line:end_line][j], pro) for j in range(process_num)]
                    results = ray.get(futures)
                    if i == nested_lines_num - 2:
                        futures = [ray_preprocessing_text.remote(lines[end_line:][j], pro) for j in range(len(lines) - end_line)]
                        results = ray.get(futures)
                    
                sentences = list(chain.from_iterable(results))
                sentence_list.append(sentences)
        
        sentence_list = list(chain.from_iterable(sentence_list))
        
        with open(post, 'a', encoding='utf-8') as fp:
            fp.write("\n".join(sentence_list))

        pbar.n += 1
        pbar.refresh()
        time.sleep(0.01)

    pbar.close() 

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list)

In [ ]:
ray.shutdown()

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/machine_reading_data_for_administrative_documents_post/AIHUB_machine_reading_data_for_administrative_documents_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_machine_reading_data_for_administrative_documents.txt'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_machine_reading_data_for_administrative_documents.txt'

In [ ]:
merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path, 
                                  deduplicate_corpus_path)